# Salvar o resultado em um novo arquivo Excel com três abas: "Tabelas Consolidada", "Empresas" e "Notas Consolidada" no PROTOTIPO_database.xlsx

## 1) Consolidando tabela de anotações para Tabela MUST 1

In [ ]:
import re
import pandas as pd
from openpyxl import load_workbook
import os

def substituir_aba_excel(df_novo, caminho_arquivo, nome_aba, engine='openpyxl'):
    """
    Substitui uma aba específica em um arquivo Excel existente por um novo DataFrame.
    
    Args:
        df_novo: DataFrame do pandas que substituirá a aba existente
        caminho_arquivo: Caminho completo do arquivo Excel
        nome_aba: Nome da aba a ser substituída
        engine: Motor do Excel a ser usado (padrão: 'openpyxl')
    """
    # Verifica se o arquivo existe
    if not os.path.exists(caminho_arquivo):
        print(f"⚠️ Arquivo não encontrado: {caminho_arquivo}")
        return False
    
    try:
        # Carrega o workbook existente
        book = load_workbook(caminho_arquivo)
        
        # Remove a aba se já existir
        if nome_aba in book.sheetnames:
            del book[nome_aba]
        
        # Salva as alterações temporárias
        book.save(caminho_arquivo)
        book.close()
        
        # Adiciona o novo DataFrame na aba especificada
        with pd.ExcelWriter(caminho_arquivo, engine=engine, mode='a') as writer:
            df_novo.to_excel(writer, sheet_name=nome_aba, index=False)
        
        print(f"✅ Aba '{nome_aba}' substituída com sucesso em {caminho_arquivo}")
        return True
        
    except Exception as e:
        print(f"❌ Erro ao substituir aba: {e}")
        return False

def extrair_empresa(nome_arquivo: str) -> str:
    """
    Extrai o nome da empresa a partir do nome do arquivo.
    Exemplos:
        "saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE_minuta_recon_2025_28_final.xlsx" -> "SUL SUDESTE"
        "saida_anotacoes_CUST-2002-114-64_ CPFL_Paulista.xlsx" -> "CPFL PAULISTA"
    """
    # Remove prefixo e extensão
    base = os.path.splitext(nome_arquivo)[0]
    base = re.sub(r"^saida_anotacoes_", "", base)

    # Padrão para encontrar o código (ex: CUST-2002-114-64)
    padrao_codigo = r'^CUST-\d{4}-\d{2,4}-\d{2,3}'
    match = re.search(padrao_codigo, base)
    
    if not match:
        return "DESCONHECIDA"
    
    # Remove o código e separadores subsequentes
    empresa_raw = base[match.end():]
    empresa_raw = re.sub(r"^[\s_-]+", "", empresa_raw)  # Remove hífens/underscores iniciais

    # Remove termos indesejados e tudo após eles
    empresa_limpa = re.split(
        r"[\s_-]*(?:minuta|recon|final|202[0-9])[\s_-]*", 
        empresa_raw, 
        flags=re.IGNORECASE
    )[0]

    # Remove caracteres especiais e espaços extras
    empresa_limpa = re.sub(r"[_\s]+", " ", empresa_limpa).strip()
    return empresa_limpa.upper() if empresa_limpa else "DESCONHECIDA"

def consolidar_anotacoes(diretorio: str):
    """
    Consolida os arquivos de anotações exportados em um único Excel.
    - Só concatena arquivos com colunas iguais.
    - Filtra num_tabela = 1 (quando existir).
    - Extrai nome da empresa do arquivo.
    """
    arquivos = [f for f in os.listdir(diretorio) if f.endswith(".xlsx") and f.startswith("saida_anotacoes")]
    
    if not arquivos:
        print("⚠️ Nenhum arquivo encontrado para consolidar.")
        return
    
    dataframes = []
    empresas = set()
    colunas_padrao = None

    for arq in arquivos:
        caminho = os.path.join(diretorio, arq)
        try:
            df = pd.read_excel(caminho)

            # Filtra apenas num_tabela = 1
            if "num_tabela" in df.columns:
                df = df[df["num_tabela"] == 1]

            # Garante colunas iguais
            if colunas_padrao is None:
                colunas_padrao = list(df.columns)
                print(f"📊 Colunas padrão definidas a partir de {arq}: {colunas_padrao}")
            elif list(df.columns) != colunas_padrao:
                print(f"⏭️ Ignorando {arq} pois as colunas não batem com o padrão.")
                continue

            # Extrai empresa
            empresa = extrair_empresa(arq)
            df.insert(0, "EMPRESA", empresa)  # força ser a primeira coluna
            df["Arquivo_Origem"] = arq
            empresas.add(empresa)

            dataframes.append(df)

        except Exception as e:
            print(f"❌ Erro ao ler {arq}: {e}")

    if not dataframes:
        print("⚠️ Nenhum dado válido para consolidar.")
        return

    df_final = pd.concat(dataframes, ignore_index=True)

    # Exporta para Excel
    caminho_saida = os.path.join(diretorio, "export_notes_MUST_tables.xlsx")
    with pd.ExcelWriter(caminho_saida, engine="openpyxl") as writer:
        df_final.to_excel(writer, sheet_name="Notas Consolidada", index=False)
        pd.DataFrame({"Empresas": sorted(empresas)}).to_excel(writer, sheet_name="Empresas", index=False)

    print(f"✅ Consolidação concluída: {caminho_saida}")
    print(f"🔎 {len(empresas)} empresas identificadas: {sorted(empresas)}")

# Caminho da pasta onde estão os arquivos de anotações
diretorio_anotacoes = r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\anotacoes_extraidas"

# Executar consolidação de anotações
consolidar_anotacoes(diretorio_anotacoes)




📊 Colunas padrão definidas a partir de saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE_minuta_recon_2025_28_final.xlsx: ['Num_Tabela', 'Cód ONS', 'Instalação', 'Letra', 'Anotacao']
✅ Consolidação concluída: C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\anotacoes_extraidas\export_notes_MUST_tables.xlsx
🔎 6 empresas identificadas: ['CPFL PAULISTA', 'ELETROPAULO', 'JAGUARI', 'NEOENERGIA ELEKTRO', 'PIRATININGA', 'SUL SUDESTE']


## 2) Consolidados dados da tabela 1 MUST e juntando tudo para o mesmo excel database_prototipo.xlsx

In [112]:
# -*- coding: utf-8 -*-
import pandas as pd
import re

# -----------------------------
# Caminho do arquivo Excel
# -----------------------------
path = r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\database\PROTOTIPO_database.xlsx"
path = r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\tabelas_extraidas\database_must.xlsx"

# -----------------------------
# Leitura das planilhas
# -----------------------------
planilha_must = pd.read_excel(path, sheet_name="Tabelas Consolidada")

# Exibe as primeiras linhas da planilha MUST
#df_tables = pd.read_excel(path, sheet_name="ANOTAÇÕES")

# Carrega a planilha MUST
df_notes = pd.read_excel(r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\anotacoes_extraidas\export_notes_MUST_tables.xlsx")


# Quando estiver pronto para substituir a aba:
substituir_aba_excel(df_notes, path, "TABELAS")

#  Verificar se foi apenas a tabela 1 no excel database
display(df_notes.head(5))
print(df_notes.shape)
df_notes["EMPRESA"].value_counts()

✅ Aba 'TABELAS' substituída com sucesso em C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\tabelas_extraidas\database_must.xlsx


,EMPRESA,Num_Tabela,Cód ONS,Instalação,Letra,Anotacao,Arquivo_Origem
0,SUL SUDESTE,1,SPUFA -138 ALTO ALEGRE - 138,kV (A),D,O atendimento ao MUST fica condicionado à manu...,saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
1,SUL SUDESTE,1,SPASS188 ASSIS I - 88,kV (A),E,"No ponto de ASSIS I 88 kV, a USUÁRIA declarou ...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
2,SUL SUDESTE,1,SPASS388 ASSIS III - 88,kV (A),F,"No ponto de ASSIS III 88 kV, a USUÁRIA declaro...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
3,SUL SUDESTE,1,SPBRB -138 BORBOREMA - 138,kV (A),P,"No ponto de BORBOREMA 138 kV, a USUÁRIA declar...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
4,SUL SUDESTE,1,SPBRP -138,BRA. PAULISTA - 138 kV (A),N,No ponto de BRAGANÇA PAULISTA 138 kV a USUÁRIA...,saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...


(210, 7)


EMPRESA
NEOENERGIA ELEKTRO    76
CPFL PAULISTA         47
ELETROPAULO           37
SUL SUDESTE           21
JAGUARI               16
PIRATININGA           13
Name: count, dtype: int64

In [ ]:


# -----------------------------
# Função para limpar código ONS
# -----------------------------
def extrair_cod_ons(valor):
    if pd.isna(valor):
        return None
    texto = str(valor).strip().upper()
    # Captura prefixo + dígito opcional (SPXXXX-138 ou SPXXXX138)
    match = re.search(r"([A-Z]{2,}[A-Z0-9]*)(?:\s*-?\s*(\d{2,3}))?", texto)
    if not match:
        return texto
    prefixo = match.group(1)  # letras + dígitos (SPASS, SPUFA, SPBRB...)
    sufixo = match.group(2)   # tensão (ex: 138, 88, etc)
    return f"{prefixo}-{sufixo}" if sufixo else prefixo

def normalizar_cod_ons(valor):
    if pd.isna(valor):
        return None
    texto = str(valor).upper().strip()
    texto = re.sub(r"\s+", "", texto)  # remove espaços internos
    
    # Detecta padrões do tipo PREFIXO + NUMERO
    match = re.match(r"([A-Z]{2,}[A-Z0-9]*?)(\d{2,4})$", texto)
    if match:
        prefixo, sufixo = match.groups()
        sufixo = str(int(sufixo))  # remove zeros à esquerda
        return f"{prefixo}-{sufixo}"
    
    # Se já tiver hífen, normaliza
    match = re.match(r"([A-Z]{2,}[A-Z0-9]*)-?(\d{2,4})$", texto)
    if match:
        prefixo, sufixo = match.groups()
        sufixo = str(int(sufixo))  # remove zeros à esquerda
        return f"{prefixo}-{sufixo}"
    
    return texto


# -----------------------------
# Aplicando a limpeza nos códigos ONS
# -----------------------------
# Padronizar códigos ONS
planilha_must["Cód ONS"] = planilha_must["Cód ONS"].apply(extrair_cod_ons).str.upper().str.strip()
df_notes["Cód ONS"] = df_notes["Cód ONS"].apply(extrair_cod_ons).str.upper().str.strip()

#! TODO: revisar se é necessário normalizar mais
# Aplica normalização ao COD ONS
#planilha_must["Cód ONS"] = planilha_must["Cód ONS"].apply(normalizar_cod_ons)
#df_notes["Cód ONS"] = df_notes["Cód ONS"].apply(normalizar_cod_ons)


print("Primeiras linhas das anotações após limpeza do código ONS:")
df_notes_filtrado = df_notes[df_notes["Num_Tabela"] == 1].reset_index(drop=True) # Filtra apenas as anotações da Tabela 1
print(df_notes_filtrado.shape)
display(df_notes_filtrado.head(7))

# -----------------------------
# Merge das tabelas com as anotações
# -----------------------------
tabela = planilha_must.merge(
    df_notes_filtrado[["Cód ONS", "Anotacao"]],
    on="Cód ONS",
    how="left"
)

# Exibe resultado final
print("\n\nTabela MUST consolidada:")
#tabela = tabela[tabela["num_tabela"] == 1].reset_index(drop=True) # Filtra apenas as anotações da Tabela 1

print(tabela.shape)
print(tabela.columns)
display(tabela)



tabela.to_excel(
    r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\AUTOMACÕES ONS\arquivos\database\must_tables_PDF_notes_merged.xlsx",
    index=False
)



tabela.to_json(
    r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\GitHub\dashboard-website-template\dashboard_must_webiste\must_tables_PDF_notes_merged.json",
    orient="records",
    force_ascii=False
)


Primeiras linhas das anotações após limpeza do código ONS:
(210, 7)


,EMPRESA,Num_Tabela,Cód ONS,Instalação,Letra,Anotacao,Arquivo_Origem
0,SUL SUDESTE,1,SPUFA-138,kV (A),D,O atendimento ao MUST fica condicionado à manu...,saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
1,SUL SUDESTE,1,SPASS188,kV (A),E,"No ponto de ASSIS I 88 kV, a USUÁRIA declarou ...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
2,SUL SUDESTE,1,SPASS388,kV (A),F,"No ponto de ASSIS III 88 kV, a USUÁRIA declaro...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
3,SUL SUDESTE,1,SPBRB-138,kV (A),P,"No ponto de BORBOREMA 138 kV, a USUÁRIA declar...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
4,SUL SUDESTE,1,SPBRP-138,BRA. PAULISTA - 138 kV (A),N,No ponto de BRAGANÇA PAULISTA 138 kV a USUÁRIA...,saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
5,SUL SUDESTE,1,SPCAT-138,kV (A),Q,"No ponto de CATANDUVA 138 kV, a USUÁRIA declar...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...
6,SUL SUDESTE,1,SPENE-138,kV (A),G,"No ponto de ENEIDA 138 kV, a USUÁRIA declarou ...",saida_anotacoes_CUST-2002-025-41 - SUL SUDESTE...




Tabela MUST consolidada:
(207, 23)
Index(['EMPRESA', 'num_tabela', 'Cód ONS', 'Tensão (kV)', 'De', 'Até',
       'Ponta 2025 Valor', 'Ponta 2025 Anotacao', 'Fora Ponta 2025 Valor',
       'Fora Ponta 2025 Anotacao', 'Ponta 2026 Valor', 'Ponta 2026 Anotacao',
       'Fora Ponta 2026 Valor', 'Fora Ponta 2026 Anotacao', 'Ponta 2027 Valor',
       'Ponta 2027 Anotacao', 'Fora Ponta 2027 Valor',
       'Fora Ponta 2027 Anotacao', 'Ponta 2028 Valor', 'Ponta 2028 Anotacao',
       'Fora Ponta 2028 Valor', 'Fora Ponta 2028 Anotacao', 'Anotacao'],
      dtype='object')


,EMPRESA,num_tabela,Cód ONS,Tensão (kV),De,Até,Ponta 2025 Valor,Ponta 2025 Anotacao,Fora Ponta 2025 Valor,Fora Ponta 2025 Anotacao,...,Fora Ponta 2026 Anotacao,Ponta 2027 Valor,Ponta 2027 Anotacao,Fora Ponta 2027 Valor,Fora Ponta 2027 Anotacao,Ponta 2028 Valor,Ponta 2028 Anotacao,Fora Ponta 2028 Valor,Fora Ponta 2028 Anotacao,Anotacao
0,SUL SUDESTE,1,SPUFA-138,138,1/Jan,31/Dez,"3,000",D,"3,500",D,...,D,"3,000",D,"3,500",D,"3,000",D,"3,500",D,O atendimento ao MUST fica condicionado à manu...
1,SUL SUDESTE,1,SPASS188,88,1/Jan,31/Dez,"44,700",E,"41,900",E,...,E,"44,700",E,"41,900",E,"44,700",E,"41,900",E,"No ponto de ASSIS I 88 kV, a USUÁRIA declarou ..."
2,SUL SUDESTE,1,SPASS388,88,1/Jan,31/Dez,"34,500",F,"35,700",F,...,F,"35,500",NaN,"35,700",F,"36,000",NaN,"35,700",F,"No ponto de ASSIS III 88 kV, a USUÁRIA declaro..."
3,SUL SUDESTE,1,SPBAST138,138,1/Jan,31/Dez,"19,800",NaN,"20,400",NaN,...,NaN,"20,400",NaN,"21,000",NaN,"20,700",NaN,"21,300",NaN,NaN
4,SUL SUDESTE,1,SPBRB-138,138,1/Jan,31/Dez,"23,900",P,"24,400",P,...,P,"23,900",P,"24,400",P,"23,900",P,"24,400",P,"No ponto de BORBOREMA 138 kV, a USUÁRIA declar..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,JAGUARI,1,SPMOC5138,5138,1/Jan,31/Dez,"5,000",NaN,"5,000",NaN,...,NaN,"5,000",NaN,"5,000",NaN,"5,000",NaN,"5,000",NaN,NaN
203,JAGUARI,1,SPOUR188,188,1/Jan,31/Dez,"29,000",NaN,"33,000",NaN,...,NaN,"29,000",NaN,"33,000",NaN,"29,000",NaN,"33,000",NaN,NaN
204,JAGUARI,1,SPOUR288,288,1/Jan,31/Dez,"69,000",NaN,"70,000",NaN,...,NaN,"69,000",NaN,"70,000",NaN,"69,000",NaN,"70,000",NaN,NaN
205,JAGUARI,1,SPSJP-138,138,1/Jan,31/Dez,"50,000",B,"50,000",B,...,B,"50,000",B,"50,000",B,"50,000",B,"50,000",B,O atendimento ao MUST fica condicionado a um f...


In [114]:
tabela["EMPRESA"].value_counts()

EMPRESA
ELETROPAULO           60
JAGUARI               39
NEOENERGIA ELEKTRO    34
CPFL PAULISTA         26
PIRATININGA           26
SUL SUDESTE           22
Name: count, dtype: int64

## Tratamento de erros

In [115]:
# Códigos nas notas que não existem na tabela MUST
nao_casados = df_notes_filtrado[~df_notes_filtrado["Cód ONS"].isin(planilha_must["Cód ONS"])]
print("Códigos de notas sem correspondência na tabela MUST:")
display(nao_casados[["Cód ONS", "Anotacao", "EMPRESA"]])

# Códigos na MUST que não existem nas notas
nao_casados2 = planilha_must[~planilha_must["Cód ONS"].isin(df_notes_filtrado["Cód ONS"])]
print("Códigos da tabela MUST sem correspondência nas notas:")
display(nao_casados2[["Cód ONS", "num_tabela"]])


Códigos de notas sem correspondência na tabela MUST:


,Cód ONS,Anotacao,EMPRESA
10,SPM,"No ponto de MARTINÓPOLIS 88 kV, a USUÁRIA decl...",SUL SUDESTE
11,SPPRP2138,"No ponto de P. PRUDENTE 2 138 kV, a USUÁRIA de...",SUL SUDESTE
12,SPPRP188,"No ponto de PRE. PRESIDENTE 1 88 kV, a USUÁRIA...",SUL SUDESTE
13,SPPRP4138,"No ponto de PRE. PRESIDENTE 4 138 kV, a USUÁRI...",SUL SUDESTE
14,SPPRP5138,"No ponto de PRE. PRUDENTE 5 -138 kV, a USUÁRIA...",SUL SUDESTE
...,...,...,...
159,SPBAN,MUST reduzidos de forma NÃO ONEROSA (limite at...,ELETROPAULO
160,SPBAN,MUST reduzidos de forma NÃO ONEROSA (limite at...,ELETROPAULO
161,SPBAN,O atendimento aos valores de MUST solicitados ...,ELETROPAULO
192,SPRRF-88,O atendimento aos valores de MUST solicitados ...,ELETROPAULO


Códigos da tabela MUST sem correspondência nas notas:


,Cód ONS,num_tabela
3,SPBAST138,1
6,SPBPER138,1
7,SPCDM-88,1
8,SPCPV-138,1
10,SPDEC-88,1
12,SPFLP-69,1
16,SPIBM-88,1
18,SPMARA88,1
19,SPAJIV138,1
24,SPAUX-138,1


## Modelagem banco de dados

In [116]:
#!pip install flask sqlalchemy flask_sqlalchemy flask_migrate flask_cors pandas

In [117]:
# Classes: DataBaseSQL, DbController, ProcessadorPandas
# Mantém nomes das colunas exatamente como estão no DataFrame.
# Classes corrigidas — apenas alteração: usar sqlalchemy.text() para executar SQL DDL
import re
import pandas as pd
from sqlalchemy import create_engine, text
from typing import List

class DataBaseSQL:
    def __init__(self, caminho_db: str = "must.db", echo: bool = False):
        self.caminho_db = caminho_db
        self.engine = create_engine(f"sqlite:///{self.caminho_db}", echo=echo)
    
    def salvar_dataframe(self, nome_tabela: str, df: pd.DataFrame, if_exists: str = "replace", index: bool = False, index_label: str = None):
        df.to_sql(nome_tabela, self.engine, if_exists=if_exists, index=index, index_label=index_label)
        print(f"[DB] Gravado: tabela '{nome_tabela}' ({len(df)} linhas).")
    
    def ler_sql(self, query: str) -> pd.DataFrame:
        return pd.read_sql(query, self.engine)
    
    def executar_sql(self, sql: str):
        # usa text() para garantir compatibilidade com SQLAlchemy
        try:
            with self.engine.begin() as conn:
                conn.execute(text(sql))
            print("[DB] SQL executado.")
        except Exception as e:
            print("[DB] ERRO ao executar SQL:", e)
            raise

class DbController(DataBaseSQL):
    def salvar_raw_original(self, nome_raw: str, df_raw: pd.DataFrame):
        self.salvar_dataframe(nome_raw, df_raw, if_exists="replace", index=False)
    
    def criar_tabela_points_com_pk_codons(self):
        sql_drop = "DROP TABLE IF EXISTS points;"
        self.executar_sql(sql_drop)
        
        sql_create = """
        CREATE TABLE points (
        "Cód ONS" TEXT PRIMARY KEY,
        "EMPRESA" TEXT,
        "num_tabela" INTEGER,
        "Tensão (kV)" INTEGER,
        "De" TEXT,
        "Até" TEXT,
        "Anotacao" TEXT,
        "original_index" INTEGER
        );
        """
        self.executar_sql(sql_create)

    
    def inserir_points(self, df_points: pd.DataFrame):
        df_points.to_sql('points', self.engine, if_exists='append', index=False)
        print(f"[DB] Inseridas {len(df_points)} linhas em 'points' (append).")
    
    def criar_tabela_tariffs_com_fk(self):
        sql = """
        CREATE TABLE IF NOT EXISTS tariffs (
          "Cód ONS" TEXT,
          year INTEGER,
          season TEXT,
          valor REAL,
          anotacao TEXT,
          FOREIGN KEY("Cód ONS") REFERENCES points("Cód ONS")
        );
        """
        self.executar_sql(sql)
    
    def inserir_tariffs(self, df_tariffs: pd.DataFrame):
        if df_tariffs.empty:
            print("[DB] Nenhuma tarifa para inserir.")
            return
        df_tariffs.to_sql('tariffs', self.engine, if_exists='append', index=False)
        print(f"[DB] Inseridas {len(df_tariffs)} linhas em 'tariffs' (append).")
    
    def criar_tabela_files_com_fk(self):
        sql = """
        CREATE TABLE IF NOT EXISTS files (
          "Cód ONS" TEXT,
          filename TEXT,
          original_index INTEGER,
          FOREIGN KEY("Cód ONS") REFERENCES points("Cód ONS")
        );
        """
        self.executar_sql(sql)
    
    def inserir_files(self, df_files: pd.DataFrame):
        if df_files.empty:
            print("[DB] Nenhum arquivo para inserir.")
            return
        df_files.to_sql('files', self.engine, if_exists='append', index=False)
        print(f"[DB] Inseridas {len(df_files)} linhas em 'files' (append).")
    
    def salvar_tabela_aprovacao(self, df_aprov: pd.DataFrame):
        self.salvar_dataframe("TABELA_APROVACAO", df_aprov, if_exists="replace", index=False)
    
    def salvar_tabela_empresas(self, df_emp: pd.DataFrame):
        df_emp.to_sql("TABELA_DE_EMPRESAS", self.engine, if_exists="replace", index_label='id')
        print("[DB] Gravado: TABELA_DE_EMPRESAS.")
    
    def criar_aprovacao_agente(self, df_rel: pd.DataFrame):
        if df_rel.empty:
            print("[DB] Nenhuma relação APROVACAO_AGENTE a gravar.")
            return
        df_rel.to_sql("APROVACAO_AGENTE", self.engine, if_exists="replace", index=False)
        print("[DB] Gravado: APROVACAO_AGENTE.")


class ProcessadorPandas:
    def __init__(self, tabela: pd.DataFrame):
        self._orig = tabela.copy().reset_index(drop=True)
        self._df = self._orig.copy()
        self._df['__originalIndex'] = self._df.index
        print(f"[PANDAS] Inicializado com shape: {self._df.shape}")
    
    def gerar_raw_para_salvar(self) -> pd.DataFrame:
        raw = self._df.copy()
        raw = raw.rename(columns={'__originalIndex': 'id'})
        return raw
    
    def gerar_df_points_para_insercao(self) -> pd.DataFrame:
        dfp = self._df[[
            'EMPRESA','num_tabela','Cód ONS','Tensão (kV)','De','Até','Anotacao','__originalIndex'
        ]].copy()
        dfp = dfp.rename(columns={'__originalIndex':'original_index'})
        return dfp
    
    def mapear_originalindex_para_codons(self, dfp: pd.DataFrame) -> dict:
        return dfp.set_index('original_index')['Cód ONS'].to_dict()
    
    def normalizar_tarifas_para_insercao(self, mapa_originalindex_para_codons: dict) -> pd.DataFrame:
        cols = self._df.columns.tolist()
        pattern = re.compile(r'^(Ponta|Fora Ponta)\s+(\d{4})\s+(Valor|Anotacao)$', re.IGNORECASE)
        linhas = []
        for col in cols:
            m = pattern.match(col)
            if not m:
                continue
            season_raw, year, kind = m.groups()
            season = 'Ponta' if season_raw.lower().startswith('ponta') else 'Fora Ponta'
            if kind.lower() == 'valor':
                val_col = col
                ann_col = f"{season_raw} {year} Anotacao"
                for idx, row in self._df.iterrows():
                    tem_val = pd.notna(row.get(val_col))
                    tem_ann = ann_col in self._df.columns and pd.notna(row.get(ann_col))
                    if not (tem_val or tem_ann):
                        continue
                    raw_val = row.get(val_col)
                    valor = None
                    if pd.notna(raw_val):
                        s = str(raw_val).replace('.', '').replace(',', '.')
                        try:
                            valor = float(s)
                        except:
                            valor = None
                    anot = row.get(ann_col) if ann_col in self._df.columns else None
                    codons = mapa_originalindex_para_codons.get(idx)
                    linhas.append({
                        'Cód ONS': codons,
                        'year': int(year),
                        'season': 'Ponta' if season == 'Ponta' else 'Fora Ponta',
                        'valor': valor,
                        'anotacao': anot if pd.notna(anot) else None
                    })
        df_tariffs = pd.DataFrame(linhas)
        df_tariffs = df_tariffs[df_tariffs['Cód ONS'].notna()].copy()
        return df_tariffs
    
    def gerar_df_files_para_insercao(self, mapa_originalindex_para_codons: dict) -> pd.DataFrame:
        def nome_arquivo(row):
            empresa = str(row.get('EMPRESA','UNKNOWN')).strip()
            chave = re.sub(r'\s+','_',empresa)
            chave = re.sub(r'[^\w\d_]','',chave)
            return f"MUST_{chave}_{int(row['__originalIndex'])}.pdf"
        df_files = pd.DataFrame({
            'original_index': self._df['__originalIndex'],
            'filename': self._df.apply(nome_arquivo, axis=1)
        })
        df_files['Cód ONS'] = df_files['original_index'].map(mapa_originalindex_para_codons)
        df_files = df_files[df_files['Cód ONS'].notna()].copy()
        df_files = df_files[['Cód ONS','filename','original_index']]
        return df_files
    
    def criar_tabela_aprovacao_df(self) -> pd.DataFrame:
        linhas = [
            ('APROVADO', 'MUST aprovado sem ressalvas'),
            ('RESSALVAS', 'MUST condicionado à obras'),
            ('LIMITADO', 'MUST limitado à um valor menor do que foi solicitado'),
            ('REPROVADO', 'Impossibilidade de atender ao MUST')
        ]
        return pd.DataFrame(linhas, columns=['id_aprovacao','id_aprovacao_detalhe'])
    
    def criar_tabela_empresas_df(self) -> pd.DataFrame:
        agentes = ['SUL E SUDESTE','JAGUARI','CPFL PAULISTA','PIRATININGA','NEOENERGIA ELEKTRO','ELETROPAULO']
        return pd.DataFrame({'id_agente': agentes})
    
    def criar_relacao_aprovacao_agente_df(self, mapeamentos: List[tuple]) -> pd.DataFrame:
        rows = []
        for apv, ag in mapeamentos:
            rows.append({'id_aprovacao': apv, 'id_agente': ag})
        return pd.DataFrame(rows)


# -------------------------
# Função orquestradora (exemplo de uso)
# -------------------------
def run_modelagem_banco_SQL(tabela: pd.DataFrame, caminho_db: str = "must.db"):
    """
    Orquestra o processamento e gravação usando classes.
    NÃO RENOMEIA NENHUMA COLUNA original do DataFrame 'tabela'.
    """
    proc = ProcessadorPandas(tabela)
    db = DbController(caminho_db, echo=False)
    
    # 1) salva raw (tabela original preservada)
    raw_para_salvar = proc.gerar_raw_para_salvar()
    nome_raw = "Tabela MUST consolidada"  # nome da tabela raw no DB
    db.salvar_raw_original(nome_raw, raw_para_salvar)
    
    # 2) salvar TABELA_DE_EMPRESAS e TABELA_APROVACAO fixas
    df_emp = proc.criar_tabela_empresas_df()
    db.salvar_tabela_empresas(df_emp)
    df_aprov = proc.criar_tabela_aprovacao_df()
    db.salvar_tabela_aprovacao(df_aprov)
    
    # 3) criar tabela points com PK "Cód ONS" e inserir dados
    db.criar_tabela_points_com_pk_codons()
    df_points = proc.gerar_df_points_para_insercao()
    db.inserir_points(df_points)
    
    # 4) mapear original_index -> Cód ONS
    mapa = proc.mapear_originalindex_para_codons(df_points)
    
    # 5) normalizar tarifas e salvar (usa "Cód ONS" como FK)
    db.criar_tabela_tariffs_com_fk()
    df_tariffs = proc.normalizar_tarifas_para_insercao(mapa)
    db.inserir_tariffs(df_tariffs)
    
    # opcional: criar tabelas por ano (se quiser)
    if not df_tariffs.empty:
        anos = sorted(df_tariffs['year'].unique())
        for y in anos:
            nome = f"tariffs_{y}"
            df_tariffs[df_tariffs['year']==y].to_sql(nome, db.engine, if_exists='replace', index=False)
            print(f"[DB] Gravado: tabela por ano '{nome}' ({len(df_tariffs[df_tariffs['year']==y])} linhas).")
    
    # 6) criar tabela files e inserir
    db.criar_tabela_files_com_fk()
    df_files = proc.gerar_df_files_para_insercao(mapa)
    db.inserir_files(df_files)
    
    # 7) exemplo: criar mapeamento APROVACAO_AGENTE (você pode alterar mapeamentos)
    mapeamentos_exemplo = [
        ('APROVADO','CPFL Jaguari'),
        ('RESSALVAS','CPFL Paulista'),
        ('LIMITADO','CPFL Piratininga'),
        ('REPROVADO','Neoenergia Elektro')
    ]
    df_rel = proc.criar_relacao_aprovacao_agente_df(mapeamentos_exemplo)
    db.criar_aprovacao_agente(df_rel)
    
    print("[MAIN] Processo finalizado. DB:", db.caminho_db)


# -------------------------
# COMO USAR:
# 1) Certifique-se que exista um DataFrame 'tabela' carregado com as 23 colunas (não modifique seus nomes).
# 2) Execute: run_modelagem_banco_SQL(tabela, caminho_db="must.db")
# -------------------------
run_modelagem_banco_SQL(tabela, caminho_db="must.db")

[PANDAS] Inicializado com shape: (207, 24)
[DB] Gravado: tabela 'Tabela MUST consolidada' (207 linhas).
[DB] Gravado: TABELA_DE_EMPRESAS.
[DB] Gravado: tabela 'TABELA_APROVACAO' (4 linhas).
[DB] SQL executado.
[DB] SQL executado.


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: points.Cód ONS
[SQL: INSERT INTO points ("EMPRESA", num_tabela, "Cód ONS", "Tensão (kV)", "De", "Até", "Anotacao", original_index) VALUES (?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [('SUL SUDESTE', 1, 'SPUFA-138', 138, '1/Jan', '31/Dez', 'O atendimento ao MUST fica condicionado à manutenção de fator de potência mínimo de 0,95 e a não ocorrência de simultaneidade  dos valores de confiab ... (101 characters truncated) ...  Prudente C1 e C2, principalmente no período da entressafra da cana - de-açúcar e em condições de despachos reduzidos nas usinas do rio Paranapanema.', 0), ('SUL SUDESTE', 1, 'SPASS188', 88, '1/Jan', '31/Dez', 'No ponto de ASSIS I 88 kV, a USUÁRIA declarou os montantes de 47,800 MW (ponta) e 44,800 MW (fora ponta) para 2026, 51,300 MW  (ponta) e 48,100 MW (f ... (504 characters truncated) ... to Grande C1 e C2, principalmente no período da entressafra da cana -de-açúcar e em condições de despachos reduzid os